In [1]:
#Import libraries (please check whether you have installed these libraries)
import numpy as np
import pandas as pd
import pickle

## Multiple Output Model for predicting the apparent quantum yields of PPRIs 
## photochemically generated by DOM

In [2]:
#Define the Multiple output model which can simultaneously predict the lnΦ3DOM* ,lnΦ1O2 and lnΦ·OH
class MultipleOutputModel():

    #Load the developed models     
    def __init__(self):
        with open('Triplet.pickle', 'rb') as e:
            self.model_tri = pickle.load(e)
        with open('Tri-singlet.pickle', 'rb') as f:
            self.model_tri_sin = pickle.load(f)
        with open('Tri-hydroxyl.pickle', 'rb') as g:
            self.model_tri_hyd = pickle.load(g)
        with open('Singlet.pickle', 'rb') as h:
            self.model_sin = pickle.load(h)
        with open('Hydroxyl.pickle', 'rb') as i:
            self.model_hyd = pickle.load(i)
    
    #Make prediction
    def predict(self, x, regressorchain = False, export=False):
        newx = x.copy()
        x1 = newx.iloc[:,:19]
        x2 = pd.concat([newx.iloc[:,:13],newx.iloc[:,19:24]],axis=1)
        x3 = pd.concat([newx.iloc[:,:13],newx.iloc[:,24:]],axis=1)
        #regressorchain: default=False. Whether the regressorchain is used when predicting lnΦ1O2 and lnΦ·OH. 
        #If True, MultipleOutput model will apply the developed chain models to predict lnΦ1O2 and lnΦ·OH.
        if regressorchain == True: 
            x2['Predicted ln3DOM'] = self.model_tri.predict(x1)
            x3['Predicted ln3DOM'] = self.model_tri.predict(x1)
            mult_x=[self.model_tri.predict(x1),self.model_tri_sin.predict(x2),self.model_tri_hyd.predict(x3)]
        else:
            mult_x=[self.model_tri.predict(x1),self.model_sin.predict(x2),self.model_hyd.predict(x3)]   
            
        df_x = pd.DataFrame(mult_x,index=['Pred lnΦ3DOM*' ,'Pred lnΦ1O2', 'Pred lnΦ·OH'])
        #export: default=False. Whether the predicted lnΦPPRIs is exported after the prediction.
        #If True, the predicted data will be exported into an Excel file.
        if export == True:
            df_x.T.to_excel('predicted AQYs.xlsx')#you can design your path to export the Excel file.
            print('predicted AQYs.xlsx is exported')
            display(df_x.T) 
            return mult_x
        else:
            display(df_x.T)
            return mult_x
    
    #Calculate R2 and RMSE for each lnΦPPRIs, if you have already calculated the observed lnΦPPRIs through photochemical experiments
    def mult_reg_score(self, true_y, pred_y, export = False):
        true_y1 = list(np.array(true_y.T))
        mult_r = []
        mult_rmse = []
        for i in range(len(pred_y)):
            y_mean = np.mean(true_y1[i])
            sse = sum((true_y1[i] - pred_y[i])**2)
            sst = sum((true_y1[i] - y_mean)**2) 
            r2 = 1 - (sse/sst)
            mult_r.append(r2)
            rmse = np.mean((true_y1[i] - pred_y[i])**2) **0.5
            mult_rmse.append(rmse)
        mult_result = pd.DataFrame({'R2':mult_r,'RMSE':mult_rmse},index=['lnΦ3DOM*' ,'lnΦ1O2', 'lnΦ·OH'])
        #export: default=False. Whether the calculated R2 and RMSE is exported.
        #If True, he calculated R2 and RMSE will be exported into an Excel file.
        if export == True:
            mult_result = pd.to_excel('mult_result.xlsx')#you can design your path to export the Excel file.
            print('mult_result.xlsx is exported')
            return mult_result
        return mult_result

In [3]:
#Import the example data from a previous literature
#The data you prepared must sort by the feature order in the example data and 
#the feature should be converted into the uniform units and calculation
feature = pd.read_excel('Example data.xlsx',sheet_name=0)#read the first sheet of file Example data.xlsx
feature.describe(include='all')#Statistical description of the example data

,DOM Category,Isolation way,Light Source,Wavelength Range,Actinometry,Temperature,pH,DOC,SUVA254,E2/E3,...,FFA Irradiation Time (h),k1O2 FFA (10-8 M-1 s-1),kd (10-5 s-1),Correction for ·OH,Probe type,Probe (mM),Probe Irradiation Time (h),kProbe ·OH (10-9 M-1 s-1),Yield,Correction for DOC
count,22.0,22.0,22.0,22.0,22.0,22.0,22.000000,22.000000,22.000000,22.000000,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,2.200000e+01,2.200000e+01,22.0
mean,3.0,2.0,2.0,0.0,2.0,25.0,7.536818,7.865455,2.073714,6.526827,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.209475,2.857009,0.413026,0.564145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.090796e-16,1.136349e-16,0.0
min,3.0,2.0,2.0,0.0,2.0,25.0,7.290000,4.260000,1.238100,5.692300,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0
25%,3.0,2.0,2.0,0.0,2.0,25.0,7.365000,5.925000,1.965050,6.109850,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0
50%,3.0,2.0,2.0,0.0,2.0,25.0,7.510000,7.270000,2.122800,6.451050,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0
75%,3.0,2.0,2.0,0.0,2.0,25.0,7.692500,9.007500,2.384575,7.055900,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0
max,3.0,2.0,2.0,0.0,2.0,25.0,8.170000,13.020000,2.601600,7.428600,...,4.0,1.0,2.5,0.0,2.0,1.0,4.0,4.400000e+00,3.500000e-01,0.0


In [4]:
#Instantiating the Multiple Output Model
model = MultipleOutputModel()

In [5]:
#Use the model to predict the lnΦPPRIs data based on the feature from example data
predicted_y = model.predict(feature,regressorchain=False,export=False)
#If you want to apply the regressorchain, switch 'regressorchain' to True;
#If you want to export the predicted data, switch 'export' to True.

,Pred lnΦ3DOM*,Pred lnΦ1O2,Pred lnΦ·OH
0,0.080488,1.125889,0.955194
1,0.396759,1.440000,1.160566
2,0.213709,0.894140,0.890409
3,0.638250,1.347493,1.301977
4,0.372110,1.099129,0.726128
5,0.434314,1.266735,0.819762
6,0.340646,1.075898,0.614667
7,0.190882,1.099920,0.758955
8,0.369978,1.398864,0.860238
9,0.076260,1.296262,0.788862


In [6]:
#If you have already calculated the ΦPPRIs data and want to explore predictive performance of the developed models
#you can also import the observed ΦPPRIs data to calculate the R2 and RMSE, but remember to transform ΦPPRIs data into lnΦPPRIs data。
target = pd.read_excel('Example data.xlsx',sheet_name=1)
lntarget = target.apply(np.log)
results = model.mult_reg_score(lntarget,predicted_y,export=False)
results

,R2,RMSE
lnΦ3DOM*,0.808798,0.149046
lnΦ1O2,0.898186,0.170861
lnΦ·OH,0.706252,0.160388


In [7]:
#You can try using the regressorchain model to make prediction and see the difference of the R2 and RMSE
predicted_y1 = model.predict(feature,regressorchain=True,export=False)
results1 = model.mult_reg_score(lntarget,predicted_y1,export=False)
results1

,Pred lnΦ3DOM*,Pred lnΦ1O2,Pred lnΦ·OH
0,0.080488,1.106643,0.818504
1,0.396759,1.405706,1.143154
2,0.213709,0.949670,0.887120
3,0.638250,1.456832,1.204934
4,0.372110,1.041088,0.768979
5,0.434314,1.255555,0.832681
6,0.340646,1.093025,0.619478
7,0.190882,1.071002,0.598532
8,0.369978,1.303976,0.733383
9,0.076260,1.198848,0.764440


,R2,RMSE
lnΦ3DOM*,0.808798,0.149046
lnΦ1O2,0.892404,0.175646
lnΦ·OH,0.723627,0.155573
